In [6]:
import numpy as np
import copy
from scipy.linalg import eigh, expm

In [2]:
# pauli matrices
pauli = np.array([np.array([[1,0],[0,1]]), np.array([[0,1],[1,0]]), np.array([[0,-1.j],[1.j,0]]), np.array([[1,0],[0,-1]])])
pauli_tensor = np.array([[np.kron(pauli[i], pauli[j]) for i in range(4)] for j in range(4)])

# building operators 
def kronecker_pad(matrix, num_qubits, starting_site): 
    kron_list = [np.eye(2) for i in range(num_qubits)]    
    kron_list[starting_site] = matrix
    if matrix.shape[0] == 4: 
        del kron_list[starting_site+1]
    
    padded_matrix = kron_list[0]
    for i in range(1, len(kron_list)):
        padded_matrix = np.kron(kron_list[i], padded_matrix)    
    return padded_matrix

# models
def heisenberg(num_qubits, bias_coeff=1.0, x_hopping_coeff=1.0, y_hopping_coeff=1.0, z_hopping_coeff=1.0): 
    terms = []
    for i in range(num_qubits): 
        bias = bias_coeff*kronecker_pad(pauli[3], num_qubits, i)
        terms.append(bias)
        
    for i in range(num_qubits-1): 
        z_hop = z_hopping_coeff*kronecker_pad(pauli_tensor[(3,3)], num_qubits, i)
        terms.append(z_hop)
        y_hop = y_hopping_coeff*kronecker_pad(pauli_tensor[(2,2)], num_qubits, i)
        terms.append(y_hop)
        x_hop = x_hopping_coeff*kronecker_pad(pauli_tensor[(1,1)], num_qubits, i)
        terms.append(x_hop)
    
    return sum(terms)

# used for initial guesses
def basis_state(num_qubits, i): 
    state = np.zeros(2**num_qubits)
    state[i] = 1.0 
    return state

In [3]:
ham = heisenberg(4)
evals, evecs = np.linalg.eigh(ham)
evals

array([-6.46410162, -5.82842712, -3.82842712, -3.        , -1.82842712,
       -1.        , -1.        , -0.17157288,  0.46410162,  1.        ,
        1.        ,  1.82842712,  3.        ,  3.82842712,  5.        ,
        7.        ])

In [37]:
def correction_state(ham, energy, state, tau=0.01):
    #op = expm(-tau*(ham-energy))
    #correction_state = op @ state
    correction_state = ham @ state - energy*state
    return correction_state / np.linalg.norm(correction_state)
    
def qdavidson_iter(ham, basis_set, tol=0.5):
    num_basis = len(basis_set)
    eff_H = np.eye(num_basis, dtype=complex)
    eff_S = np.eye(num_basis, dtype=complex)
    for i in range(num_basis): 
        for j in range(num_basis): 
            eff_H[i][j] = basis_set[i].conj().T @ ham @ basis_set[j]
            eff_S[i][j] = basis_set[i].conj().T @ basis_set[j]
            
    evals, evecs = eigh(eff_H, eff_S)
    estates = [np.array(sum([evecs[:,i][j] * basis_set[j] for j in range(num_basis)])) for i in range(num_basis)]
        
    new_basis_set = copy.deepcopy(basis_set)
    residue_vals = []
    for i in range(num_basis): 
        val = np.linalg.norm((ham @ estates[i]) - (evals[i] * estates[i]))
        residue_vals.append(val)
        if val > tol: 
            state = correction_state(ham, evals[i], estates[i])
            #print(state)
            if linear_independence(state, basis_set, eff_S, tol): 
                new_basis_set.append(state)
            
    return evals, estates, residue_vals, new_basis_set

def linear_independence(correction_vec, basis_set, eff_S, tol=0.5): 
    b = np.array([correction_vec.conj().T @ basis_set[i] for i in range(len(basis_set))])
    return np.linalg.norm(np.linalg.pinv(eff_S) @ b) < tol    

def qdavidson(ham, initial_basis_set, num_iter, tol=0.5): 
    basis_set = copy.deepcopy(initial_basis_set)
    for i in range(num_iter): 
        evals, estates, residue_vals, basis_set = qdavidson_iter(ham, basis_set, tol)
    return basis_set, evals

In [38]:
num_qubits = 4
ham = heisenberg(num_qubits)
basis_set = [basis_state(num_qubits, i) for i in range(4)]
evals, estates, residue_vals, new_basis_set = qdavidson_iter(ham, basis_set)

In [39]:
evals

array([-0.23606798,  1.        ,  4.23606798,  7.        ])

In [40]:
residue_vals

[1.7013016167040798, 2.0, 1.051462224238267, 0.0]

In [41]:
new_basis_set

[array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([ 0.00000000e+00+0.j, -1.71300339e-16+0.j, -9.78859081e-17+0.j,
         0.00000000e+00+0.j, -1.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j]),
 array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 array([ 0.00000000e+00+0.j,  4.22353937e-16+0.j, -4.22353937e-16+0.j,
         0.00000000e+00+0.j, -1.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j,  

In [42]:
estates

[array([ 0.        +0.j,  0.52573111+0.j, -0.85065081+0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j]),
 array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 array([ 0.        +0.j, -0.85065081+0.j, -0.52573111+0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j]),
 array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])]

In [43]:
ham @ estates[0]

array([ 0.        +0.j, -0.12410828+0.j,  0.20081142+0.j,  0.        +0.j,
       -1.70130162+0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j])

In [44]:
ham @ estates[0] - evals[0] * estates[0]

array([ 0.00000000e+00+0.j, -2.91433544e-16+0.j, -1.66533454e-16+0.j,
        0.00000000e+00+0.j, -1.70130162e+00+0.j,  0.00000000e+00+0.j,
        0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        0.00000000e+00+0.j])

In [45]:
estates[0]

array([ 0.        +0.j,  0.52573111+0.j, -0.85065081+0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j])